In [1]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.activation import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam, RMSprop
from keras.datasets import fashion_mnist

import numpy as np
import matplotlib.pyplot as plt


KeyboardInterrupt: 

In [ ]:
def build_discriminator():
    discriminator = Sequential()
    discriminator.add(Conv2D(32, kernel_size=3, strides=2, input_shape=(28, 28, 1), padding="same"))
    discriminator.add(LeakyReLU(alpha=0.2))
    discriminator.add(Dropout(0.25))
    discriminator.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    discriminator.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    discriminator.add(BatchNormalization(momentum=0.8))
    discriminator.add(LeakyReLU(alpha=0.2))
    discriminator.add(Dropout(0.25))
    discriminator.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    discriminator.add(BatchNormalization(momentum=0.8))
    discriminator.add(LeakyReLU(alpha=0.2))
    discriminator.add(Dropout(0.25))
    discriminator.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    discriminator.add(BatchNormalization(momentum=0.8))
    discriminator.add(LeakyReLU(alpha=0.2))
    discriminator.add(Dropout(0.25))
    discriminator.add(Flatten())
    discriminator.add(Dense(1, activation='sigmoid'))
    discriminator.summary()
    img = Input(shape=(28, 28, 1))
    probability = discriminator(img)
    return Model(inputs=img, outputs=probability)

In [ ]:
def build_generator():
    generator = Sequential()
    generator.add(Dense(64 * 28 * 28, activation="relu", input_dim=100))
    generator.add(Reshape((28, 28, 64)))
    generator.add(Conv2D(64, kernel_size=3, padding="same", strides=(1,1),
                         activation="relu"))
    generator.add(BatchNormalization(momentum=0.8))
    generator.add(Conv2D(32, kernel_size=3, padding="same",
                         activation="relu"))
    generator.add(BatchNormalization(momentum=0.8))
    generator.add(Conv2D(1, kernel_size=3, padding="same",
                         activation="relu"))
    generator.summary()
    noise = Input(shape=(100,))
    fake_image = generator(noise)
    return Model(inputs=noise, outputs=fake_image)

build_generator()

In [ ]:
def visualize_input(img, ax):
    ax.imshow(img, cmap='gray')
    width, height = img.shape
    thresh = img.max() / 2.5
    for x in range(width):
        for y in range(height):
            ax.annotate(str(round(img[x][y], 2)), xy=(y, x),
                        horizontalalignment='center',
                        verticalalignment='center',
                        color='white' if img[x][y] < thresh else 'black')

In [ ]:
def train(epochs, discriminator_epochs, batch_size=64, save_interval=50):
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for epoch in range(discriminator_epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)
        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        print("%d [D loss: %f, acc.: %.2f%%] " % (epoch, d_loss[0], 100 * d_loss[1]))
    for epoch in range(epochs):
        # Train the combined network (Generator)
        noise = np.random.normal(0, 1, (batch_size, 100))
        g_loss = combined.train_on_batch(noise, valid)
        print("%d [G loss: %f]" % (epoch, g_loss))
        if epoch % save_interval == 0:
            plot_generated_images(epoch, generator)

In [ ]:
def train(epochs, batch_size=64, save_interval=50):
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    for epoch in range(epochs):
        # Train Discriminator network
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)
        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        plt.figure(figsize=(10,10))
        dim = (10,10)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        # Train the combined network (Generator)
        g_loss = combined.train_on_batch(noise, valid)
        print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss))
        if epoch % save_interval == 0:
            for i in range(gen_imgs.shape[0]):
                plt.subplot(dim[0], dim[1], i + 1)
                plt.imshow(gen_imgs[i], interpolation='nearest', cmap='gray_r')
                plt.axis('off')
                plt.tight_layout()
            for i in range(gen_imgs.shape[0], gen_imgs.shape[0] + imgs.shape[0]):
                plt.subplot(dim[0], dim[1], i + 1)
                plt.imshow(imgs[i-gen_imgs.shape[0]], interpolation='nearest', cmap='gray_r')
                plt.axis('off')
                plt.tight_layout()
            plt.savefig('gen_images/train_compare_image_epoch_%d.png' % epoch)
            plot_generated_images(epoch, generator)

In [ ]:
def plot_generated_images(
        epoch, generator, examples=100, dim=(10, 10),
        figsize=(10, 10)):
    latent_dim = 100
    noise = np.random.normal(0, 1, size=[examples, latent_dim])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 28, 28)
    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i + 1)
        plt.imshow(generated_images[i], interpolation='nearest',
               cmap='gray_r')
        plt.axis('off')
        plt.tight_layout()
    plt.savefig('gen_images/gan_generated_image_epoch_%d.png' % epoch)

In [ ]:
training_data = np.load('../datasets/full_numpy_bitmap_smiley face.npy')
# training_data = np.array([np.array_split(img_arr, 28) for img_arr in training_data])
training_data = training_data.reshape(training_data.shape[0], 28, 28).astype(
    "float32"
) / 127.5 - 1.0
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111)
visualize_input(training_data[0], ax)

In [ ]:
# (training_data, _), (_, _) = fashion_mnist.load_data()
# fig = plt.figure(figsize=(12, 12))
# ax = fig.add_subplot(111)
# visualize_input(training_data[0], ax)

In [ ]:
# X_train = training_data / 255.0
X_train = training_data
optimizer = Adam(learning_rate=0.001, beta_1=0.5)
disc_optimizer = Adam(learning_rate=0.0001, beta_1=0.5)
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=disc_optimizer, metrics=['accuracy'])
discriminator.trainable = False
generator = build_generator()
z = Input(shape=(100,))
img = generator(z)
valid = discriminator(img)
combined = Model(inputs=z, outputs=valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices())

In [ ]:
train(1000, 32)

In [ ]:
plot_generated_images(999999, generator)